In [ ]:
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
import os
from pipe import conf
from pipe import PipeParam , PipeControl
import time
import numpy as np

def find_file(substrings, datapath, warn=True):
            for substring in substrings:
                if substring in os.listdir(datapath):
                    direc_path = os.path.join(datapath,substring)
                    for file in os.listdir(direc_path):
                        if substring in file:
                            return os.path.join(direc_path, file)
            if warn:
                print('Warning: \"{:s}\" file not found'.format(substrings[0]))
            return None

        
# To record processing time
global_start_time = time.time()
datapath = '../home/data/user/che_data_dev/repo'#os.path.join('../','data/user/che_data_dev/repo')
conf.data_root = datapath
conf.ref_lib_data = 'Ref'


# num_obs_start = 105 error
# num_obs_end = 125

# num_obs_start = 220
# num_obs_end = 240  error in one file

# num_obs_start = 300
# num_obs_end = 310  error

# num_obs_start = 1100
# num_obs_end = 1150

num_obs_start = 1400
num_obs_end = 1401  

exists= False
observation_lst = sorted(os.listdir(datapath))[num_obs_start:num_obs_end]
#observation_lst = ['100002013301'] # Test observation which gives error

visit = 'L0.5'
star_cat_dir = 'EXT'
for obs in observation_lst:
    if obs in os.listdir('Outputs'):
        print(f'Already Output present for {obs}')
        exists = True
        pass
    else:
        start_time = time.time()
        target = obs
        output_dir = f'Outputs/{target}'

        # Make output directory
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Generate PIPE parameter file
        pps = PipeParam( target , visit,datapath=datapath,outdir=output_dir,version='1')
        #pps.nthreads = 20 # Number of threads to use ; default is
        # number of virtual CPU cores - 1.

        # These are data files used by PIPE.
        pps.file_att = find_file(("SCI_RAW_Attitude", "attitude."),datapath=os.path.join(datapath,target,visit))
        pps.file_sa_raw = find_file(("SCI_RAW_SubArray", "raw."),datapath=os.path.join(datapath,target,visit))
        pps.file_hk = find_file(("SCI_RAW_HkExtended",),datapath=os.path.join(datapath,target,visit))
        pps.file_im = find_file(("SCI_RAW_Imagette", "imagettes."),datapath=os.path.join(datapath,target,visit), warn=False)
        pps.file_starcat = find_file(("EXT_PRE_StarCatalogue", "starcat."),datapath=os.path.join(datapath,target,star_cat_dir))

        #pps.sa_range = (1000,3000)


        pps.klip = 10 # Number of principal components for PSF fit
        pps.fit_bgstars = True # This enables PSF photometry of
        # bright stars in the field that
        # are not the target
        # Generate a PIPE control object using the parameter file
        pc = PipeControl( pps )

        # This is the time - consuming step where PIPE is executed

        try:
            if __name__ ==  '__main__':
                pc.process_eigen_sa()
                end_time = time.time() - start_time
                print("Here is the total time with try section: ", end_time/60)
                with open(f'{output_dir}/total_time_min.txt', 'w') as file:
                    file.write(f'{end_time/60}\n')

                with open('Outputs/obs_time_min.txt', 'a') as file:
                    file.write(f'{obs}     {end_time/60}\n')    
        except Exception as e:
            print("Executing the Except block")
            with open(f'{output_dir}/error_in_try.txt','a') as file:
                file.write(f'{e}\n with bg and sattelite on\n')
            try:
             # Generate PIPE parameter file
                pps = PipeParam( target , visit,datapath=datapath,outdir=output_dir,version='1')
                pps.nthreads = 10 # 20 is better # Number of threads to use ; default is
                # number of virtual CPU cores - 1.

                # These are data files used by PIPE.
                pps.file_att = find_file(("SCI_RAW_Attitude", "attitude."),datapath=os.path.join(datapath,target,visit))
                pps.file_sa_raw = find_file(("SCI_RAW_SubArray", "raw."),datapath=os.path.join(datapath,target,visit))
                pps.file_hk = find_file(("SCI_RAW_HkExtended",),datapath=os.path.join(datapath,target,visit))
                pps.file_im = find_file(("SCI_RAW_Imagette", "imagettes."),datapath=os.path.join(datapath,target,visit), warn=False)
                pps.file_starcat = find_file(("EXT_PRE_StarCatalogue", "starcat."),datapath=os.path.join(datapath,target,star_cat_dir))

                #pps.sa_range = (1000,3000)
                pps.remove_satellites = False 
                pps.fit_bgstars = False


                pps.klip = 1 # Number of principal components for PSF fit #1 for faint stars, 10 for bright stars
                
                # are not the target
                # Generate a PIPE control object using the parameter file
                pc = PipeControl( pps )

                if __name__ ==  '__main__':
                    pc.process_eigen_sa()
                    end_time = time.time() - start_time
                    print("Here is the total time: ", end_time/60)
                    with open(f'{output_dir}/total_time_min.txt', 'w') as file:
                        file.write(f'{end_time/60} with Bg star off and satellite off\n')
                    with open('Outputs/obs_time_min.txt', 'a') as file:
                        file.write(f'{obs}     {end_time/60}\n')
            except Exception as f:
                with open(f'{output_dir}/error_in_try.txt','a') as file:
                    file.write(f'{f}\n with bg and sattelite off\n')


                with open('Outputs/obs_time_min.txt', 'a') as file:
                    file.write(f'{obs}     0\n')


                continue


            #pc.make_psf_lib()


global_end_time = time.time() - global_start_time    
if exists == False:
    with open('Outputs/global_time_min.txt', 'a') as file:
            file.write(f'{num_obs_end-num_obs_start}     {global_end_time}\n')    


print('Observations done')




Tue Oct 15 15:03:35 2024 [0.00 min] PIPE version 1.1
Tue Oct 15 15:03:35 2024 [0.00 min] ---------
name = 100018035604
visit = L0.5
datapath = ../home/data/user/che_data_dev/repo
version = 1
outdir = Outputs/100018035604
file_log = Outputs/100018035604/logfile.txt
calibpath = Ref
file_att = ../home/data/user/che_data_dev/repo/100018035604/L0.5/SCI_RAW_Attitude/CH_PR100018_TG035604_TU2021-05-16T08-53-00_SCI_RAW_Attitude_V0200.fits
file_sa_raw = ../home/data/user/che_data_dev/repo/100018035604/L0.5/SCI_RAW_SubArray/CH_PR100018_TG035604_TU2021-05-16T08-55-49_SCI_RAW_SubArray_V0200.fits
file_hk = ../home/data/user/che_data_dev/repo/100018035604/L0.5/SCI_RAW_HkExtended/CH_PR100018_TG035604_TU2021-05-16T08-52-47_SCI_RAW_HkExtended_V0200.fits
file_im = ../home/data/user/che_data_dev/repo/100018035604/L0.5/SCI_RAW_Imagette/CH_PR100018_TG035604_TU2021-05-16T08-55-29_SCI_RAW_Imagette_V0200.fits
file_starcat = ../home/data/user/che_data_dev/repo/100018035604/EXT/EXT_PRE_StarCatalogue/CH_PR100018_

In [4]:
num_obs_start = 800
num_obs_end = 840  


observation_lst = sorted(os.listdir(datapath))[num_obs_start:num_obs_end]

In [5]:
observation_lst

['100018004101',
 '100018004201',
 '100018004601',
 '100018005101',
 '100018005601',
 '100018005701',
 '100018007101',
 '100018007102',
 '100018007301',
 '100018007302',
 '100018007303',
 '100018007401',
 '100018007402',
 '100018007403',
 '100018007404',
 '100018007501',
 '100018007502',
 '100018007503',
 '100018007504',
 '100018007801',
 '100018007901',
 '100018007902',
 '100018008001',
 '100018008003',
 '100018008202',
 '100018008301',
 '100018008303',
 '100018008501',
 '100018009601',
 '100018009602',
 '100018009701',
 '100018010801',
 '100018010802',
 '100018010901',
 '100018010902',
 '100018011001',
 '100018011101',
 '100018011102',
 '100018011201',
 '100018012201']

In [13]:
for obs in observation_lst:
    if obs in os.listdir('Outputs'):
        pass
    else:
        print('obs')

obs
obs
obs
obs
obs
obs


In [7]:
s =1